In [1]:
! kaggle

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
! cp kaggle.json /root/.kaggle/kaggle.json

In [3]:
! kaggle

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: too few arguments


In [0]:
! chmod 600 /root/.kaggle/kaggle.json

In [5]:
! kaggle datasets download ellenyusa/4classimages

 86% 33.0M/38.5M [00:00<00:00, 23.5MB/s]
100% 38.5M/38.5M [00:00<00:00, 41.8MB/s]


In [0]:
# ! kaggle datasets download thedagger/pokemon-generation-one

In [0]:
! unzip -qq 4classimages.zip

In [0]:
! rm -rf data/data

In [0]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.applications import ResNet50

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(rotation_range=15, shear_range=.1, zoom_range=.3)

generator = gen.flow_from_directory("data/")

Found 808 images belonging to 4 classes.


In [15]:
resnet_model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(256, 256, 3))

94773248/94765736 [==============================] - 1s 0us/step


In [0]:
for layer in resnet_model.layers:
  layer.trainable = False

In [17]:
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [18]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model



in_layer = Input(shape=(256, 256, 3))
resnet_layer = resnet_model(in_layer)
flat = Flatten()(resnet_layer)
d1 = Dense(units=400, activation="tanh")(flat)
d2 = Dense(units=100, activation="tanh")(d1)
out_layer = Dense(units=4, activation="softmax")(d2)

model = Model(in_layer, out_layer)

model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet50 (Model)             (None, 8, 8, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 400)               52429200  
_________________________________________________________________
dense_4 (Dense)              (None, 100)               40100     
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 404       
Total params: 76,057,416
Trainable params: 52,469,704
Non-trainable params: 23,587,712
______________________________________

In [19]:
model.fit_generator(generator, steps_per_epoch=20, epochs=5)

Epoch 1/5
20/20 [==============================] - 9s 449ms/step - loss: 1.0589 - accuracy: 0.6932
Epoch 2/5
20/20 [==============================] - 9s 451ms/step - loss: 0.2518 - accuracy: 0.9286
Epoch 3/5
20/20 [==============================] - 9s 450ms/step - loss: 0.1944 - accuracy: 0.9383
Epoch 4/5
20/20 [==============================] - 9s 451ms/step - loss: 0.1416 - accuracy: 0.9594
Epoch 5/5
20/20 [==============================] - 9s 466ms/step - loss: 0.1123 - accuracy: 0.9703


In [40]:
! wget https://previews.123rf.com/images/piksel/piksel1301/piksel130100090/17604593-casual-young-man-standing.jpg

--2020-05-24 15:55:22--  https://previews.123rf.com/images/piksel/piksel1301/piksel130100090/17604593-casual-young-man-standing.jpg
Resolving previews.123rf.com (previews.123rf.com)... 184.30.188.57
Connecting to previews.123rf.com (previews.123rf.com)|184.30.188.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76003 (74K) [image/jpeg]
Saving to: ‘17604593-casual-young-man-standing.jpg’

17604593-casual-you 100%[===================>]  74.22K  --.-KB/s    in 0.05s   

2020-05-24 15:55:23 (1.33 MB/s) - ‘17604593-casual-young-man-standing.jpg’ saved [76003/76003]



In [0]:
img = np.array(load_img("17604593-casual-young-man-standing.jpg", target_size=(256, 256)))

In [42]:
img.shape

(256, 256, 3)

In [43]:
np.argmax(model.predict(np.array([img])), axis=1)

array([0])

In [44]:
generator.class_indices

{'Humans': 0, 'cats': 1, 'dogs': 2, 'horses': 3}